<a href="https://colab.research.google.com/github/bonguchandu123/Todoapp/blob/main/jobtrqacker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q numpy
!pip install -q qiskit qiskit-aer streamlit pyngrok
!pip install pylatexenc


In [ ]:
!pip install pandas

In [ ]:
!pip install pillow


In [ ]:
%%writefile app.py
import streamlit as st
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import partial_trace, DensityMatrix, Statevector
from qiskit.visualization import plot_bloch_vector
from qiskit.quantum_info import concurrence
from datetime import datetime
from random import choice, randint
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
import sys
import json
import time
import threading
import numpy as np


# Install required packages if missing
try:
    import pylatexenc
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pylatexenc"])
    import pylatexenc

# ------------------------------
# Page setup
# ------------------------------
st.set_page_config(page_title="IBM Quantum Jobs Dashboard", layout="wide")
st.title("IBM Quantum Jobs Tracker (Qiskit Simulator)")

# ------------------------------
# Initialize session state
# ------------------------------
if "jobs" not in st.session_state:
    st.session_state.jobs = []
if "pause_updates" not in st.session_state:
    st.session_state.pause_updates = False
if "admin_logged_in" not in st.session_state:
    st.session_state.admin_logged_in = False
if "page" not in st.session_state:
    st.session_state.page = "user_dashboard"
if "editor_circuit" not in st.session_state:
    st.session_state.editor_circuit = {"0": [], "1": []}
if "current_user" not in st.session_state:
    st.session_state.current_user = "default_user"
if "filter_user" not in st.session_state:
    st.session_state.filter_user = []
if "authenticated" not in st.session_state:
    st.session_state.authenticated = False
if "users" not in st.session_state:
    st.session_state.users = {}

def generate_quantum_password(length=8):
    qc = QuantumCircuit(length, length)
    qc.h(range(length))
    qc.measure(range(length), range(length))

    simulator = AerSimulator()
    result = simulator.run(qc, shots=1).result()
    counts = result.get_counts()

    binary = list(counts.keys())[0]
    return ''.join(['A' if b == '0' else 'Z' for b in binary])  # You can customize chars


# ------------------------------
# Constants
# ------------------------------
DEVICES = ["qasm_simulator", "statevector_simulator"]
STATUSES = ["QUEUED", "RUNNING", "COMPLETED", "FAILED"]
JOBS_PER_DEVICE = 3
simulators = {
    "qasm_simulator": AerSimulator(),
    "statevector_simulator": AerSimulator(method='statevector')
}

# Helper: compute Bloch vector from single-qubit density matrix
# ------------------------------


# ------------------------------
# Navigation Buttons
# ------------------------------
with st.sidebar:
    if st.session_state.authenticated:
        st.subheader("Navigation")
        if st.button("User Dashboard"):
            st.session_state.page = "user_dashboard"
        if st.button("Admin Dashboard"):
            st.session_state.page = "admin_login" if not st.session_state.admin_logged_in else "admin_dashboard"
        if st.button("Search Jobs"):
            st.session_state.page = "search_jobs"
        if st.button("Circuit Editor"):
            st.session_state.page = "circuit_editor"
        if st.button("Bloch Spheres"):
            st.session_state.page = "bloch_spheres"
        if st.button("Analytics & Comparison"):
            st.session_state.page = "analytics_comparison"
        if st.button("Entanglement Metrics"):
            st.session_state.page = "entanglement_metrics"
        if st.button("Job Replay"):
            st.session_state.page = "job_replay"
        if st.button("Sign Out"):
            st.session_state.authenticated = False
            st.session_state.current_user = ""
            st.rerun()



# ------------------------------
# Admin login function
# ------------------------------
def admin_login(username, password):
    return username == "chandu" and password == "1234"

# ------------------------------
# Helper: visualize results as bar chart
# ------------------------------
def plot_counts_graph(counts):
    fig, ax = plt.subplots()
    ax.bar(counts.keys(), counts.values(), color='skyblue')
    ax.set_xlabel("Measurement Outcome")
    ax.set_ylabel("Counts")
    ax.set_title("Measurement Counts Bar Graph")
    st.pyplot(fig)

# ------------------------------
# Helper: compute Bloch vector from single-qubit density matrix

def bloch_vector_from_dm(dm):
    X = np.array([[0, 1], [1, 0]])
    Y = np.array([[0, -1j], [1j, 0]])
    Z = np.array([[1, 0], [0, -1]])
    return np.array([
        np.real(np.trace(dm.data @ X)),
        np.real(np.trace(dm.data @ Y)),
        np.real(np.trace(dm.data @ Z))
    ])
# ------------------------------
# Job submission function
# ------------------------------
def submit_job(device, user="default_user", custom_circuit=None):
    if custom_circuit is None:
        qc = QuantumCircuit(2, 2)
        if choice([True, False]): qc.h(0)
        else: qc.x(0)
        if choice([True, False]): qc.cx(0, 1)
        else: qc.cz(0, 1)
        if choice([True, False]): qc.h(1)
        else: qc.x(1)
        qc.measure([0, 1], [0, 1])
    else:
        qc = custom_circuit

    job = {
        "Job ID": str(datetime.now().timestamp()),
        "User": user,
        "Device": device,
        "Status": "QUEUED",
        "Submitted At": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Circuit": qc,
        "Result": None
    }
    st.session_state.jobs.append(job)

    # Run in a separate thread to simulate progression
    def run_job(job_ref):
        time.sleep(randint(1, 3))
        job_ref["Status"] = "RUNNING"
        time.sleep(randint(2, 5))
        if choice([True, True, False]):
            simulator = simulators[device]
            result = simulator.run(job_ref["Circuit"], shots=1024).result()
            job_ref["Result"] = result.get_counts(job_ref["Circuit"])
            job_ref["Status"] = "COMPLETED"
        else:
            job_ref["Status"] = "FAILED"
            job_ref["Result"] = {}

    threading.Thread(target=run_job, args=(job,)).start()
    return job

# ------------------------------
# Generate initial mock jobs
# ------------------------------
if not st.session_state.jobs:
    for device in DEVICES:
        for _ in range(JOBS_PER_DEVICE):
            submit_job(device)


if not st.session_state.authenticated:
    st.title("Welcome to Quantum Jobs Dashboard")
    auth_choice = st.radio("Choose an option:", ["Login", "Signup"])

    if auth_choice == "Signup":
        new_user = st.text_input("Choose Username")
        custom_password = st.text_input("Choose Password (or leave blank to auto-generate)", type="password")

        if st.button("Signup"):
            if new_user in st.session_state.users:
                st.error("Username already exists!")
            elif new_user.strip() == "":
                st.error("Username cannot be empty.")
            else:
                password = custom_password if custom_password else generate_quantum_password()
                st.session_state.users[new_user] = password
                st.success(f"✅ User '{new_user}' created successfully!")
                if not custom_password:
                    st.info(f"🧪 Your Quantum Password: `{password}`")
                st.session_state.current_user = new_user
                st.session_state.authenticated = True
                st.session_state.page = "user_dashboard"
                st.rerun()

    elif auth_choice == "Login":
        username = st.text_input("Username")
        password = st.text_input("Password", type="password")

        if st.button("Login"):
            if username in st.session_state.users and st.session_state.users[username] == password:
                st.success(f"🔓 Logged in as {username}")
                st.session_state.authenticated = True
                st.session_state.current_user = username
                st.session_state.page = "user_dashboard"
                st.rerun()
            else:
                st.error("❌ Invalid username or password")

    st.stop()



# ------------------------------
# PAGE: User Dashboard
# ------------------------------
if st.session_state.page == "user_dashboard":
    st.header("User Dashboard")
    st.sidebar.subheader("Submit New Job")
    with st.sidebar.form("job_form"):
        device_choice = st.selectbox("Select Device", DEVICES)
        h0 = st.checkbox("Apply H on qubit 0")
        x0 = st.checkbox("Apply X on qubit 0")
        cx = st.checkbox("Apply CX (0->1)")
        cz = st.checkbox("Apply CZ (0->1)")
        h1 = st.checkbox("Apply H on qubit 1")
        x1 = st.checkbox("Apply X on qubit 1")
        user_name_input = st.text_input("Enter User Name", value=st.session_state.current_user)
        submitted = st.form_submit_button("Submit Job")
        if submitted:
            qc = QuantumCircuit(2,2)
            if h0: qc.h(0)
            if x0: qc.x(0)
            if h1: qc.h(1)
            if x1: qc.x(1)
            if cx: qc.cx(0,1)
            if cz: qc.cz(0,1)
            qc.measure([0,1],[0,1])
            submit_job(device_choice, user=user_name_input, custom_circuit=qc)
            st.session_state.current_user = user_name_input
            st.success(f"Job submitted by '{user_name_input}'!")

    # Filters & Refresh
    st.sidebar.subheader("Job Filters & Download")
    refresh_interval = st.sidebar.slider("Refresh interval (seconds)", 1, 10, 3)
    st.session_state.pause_updates = st.sidebar.checkbox("Pause Automatic Refresh", value=st.session_state.pause_updates)
    filter_device = st.sidebar.multiselect("Filter by Device", DEVICES, default=DEVICES)
    filter_status = st.sidebar.multiselect("Filter by Status", STATUSES, default=STATUSES)
    users_list = list(set([job["User"] for job in st.session_state.jobs]))
    filter_user = st.sidebar.multiselect("Filter by User", users_list, default=users_list)  # show all users by default
    search_job = st.sidebar.text_input("Search Job ID")

    filtered_jobs = [
        job for job in st.session_state.jobs
        if job["Device"] in filter_device
        and job["Status"] in filter_status
        and job["User"] in filter_user
        and (search_job in job["Job ID"])
    ]

    st.subheader(f"Last updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    if filtered_jobs:
        df = pd.DataFrame(filtered_jobs)
        st.table(df.drop(columns=["Circuit", "Result"]))
        csv = df.drop(columns=["Circuit", "Result"]).to_csv(index=False).encode('utf-8')
        json_data = df.drop(columns=["Circuit", "Result"]).to_dict(orient='records')
        st.sidebar.download_button("Download CSV", csv, file_name="jobs.csv", mime="text/csv")
        st.sidebar.download_button("Download JSON", json.dumps(json_data, indent=4), file_name="jobs.json", mime="application/json")

        st.subheader("Job Details & Circuit Visualization")
        for idx, job_info in enumerate(filtered_jobs):
            job_id = job_info["Job ID"]
            with st.expander(f"Job ID: {job_id} | User: {job_info['User']} | Device: {job_info['Device']} | Status: {job_info['Status']}"):
                st.write(f"Submitted At: {job_info['Submitted At']}")
                st.write(f"Status: {job_info['Status']}")
                vis_mode = st.radio("Circuit View Mode", ["mpl","text"], key=f"user_{job_id}_{idx}_{time.time()}")
                if vis_mode=="mpl":
                    st.pyplot(job_info["Circuit"].draw('mpl'))
                else:
                    st.text(job_info["Circuit"].draw('text'))
                if job_info["Result"]:
                    st.write("Measurement Histogram:")
                    st.pyplot(plot_histogram(job_info["Result"]))
                    st.write("Measurement Counts Bar Graph:")
                    plot_counts_graph(job_info["Result"])

# ------------------------------
# PAGE: Circuit Editor
# ------------------------------
elif st.session_state.page == "circuit_editor":
    st.header("Circuit Editor (Multi-Gate)")
    st.write("Qubit 0:", st.session_state.editor_circuit["0"])
    st.write("Qubit 1:", st.session_state.editor_circuit["1"])

    gate_type = st.selectbox("Select Gate", ["H", "X", "CX", "CZ"])
    qubit_index = st.selectbox("Select Qubit (0 or 1)", ["0", "1"])
    if st.button("Add Gate"):
        st.session_state.editor_circuit[qubit_index].append(gate_type)
        st.success(f"Added {gate_type} to qubit {qubit_index}")

    if st.button("Clear Circuit"):
        st.session_state.editor_circuit = {"0": [], "1": []}
        st.success("Circuit cleared")

    with st.form("circuit_submit_form"):
        device_choice = st.selectbox("Select Device", DEVICES)
        user_name = st.text_input("Enter User Name", value=st.session_state.current_user)
        submitted = st.form_submit_button("Submit Circuit Job")
    if submitted:
        qc = QuantumCircuit(2,2)
        for gate in st.session_state.editor_circuit["0"]:
            if gate=="H": qc.h(0)
            if gate=="X": qc.x(0)
        for gate in st.session_state.editor_circuit["1"]:
            if gate=="H": qc.h(1)
            if gate=="X": qc.x(1)
        if "CX" in st.session_state.editor_circuit["0"] + st.session_state.editor_circuit["1"]:
            qc.cx(0,1)
        if "CZ" in st.session_state.editor_circuit["0"] + st.session_state.editor_circuit["1"]:
            qc.cz(0,1)
        qc.measure([0,1],[0,1])
        submit_job(device_choice, user=user_name, custom_circuit=qc)
        st.session_state.current_user = user_name
        st.session_state.editor_circuit = {"0": [], "1": []}
        st.success(f"Circuit submitted as job by user '{user_name}' on device '{device_choice}'!")

# ------------------------------
# ------------------------------
# PAGE: Admin Login / Dashboard
# ------------------------------
elif st.session_state.page == "admin_login":
    st.header("Admin Login")
    username = st.text_input("Username")
    password = st.text_input("Password", type="password")
    if st.button("Login"):
        if admin_login(username, password):
            st.session_state.admin_logged_in = True
            st.session_state.page = "admin_dashboard"
            st.success("Logged in successfully!")
        else:
            st.error("Invalid credentials!")

elif st.session_state.page == "admin_dashboard" and st.session_state.admin_logged_in:
    st.header("Admin Dashboard")
    if st.session_state.jobs:
        df = pd.DataFrame(st.session_state.jobs)
        st.table(df.drop(columns=["Circuit", "Result"]))

        st.subheader("Update Job Status")
        job_ids = df["Job ID"].tolist()
        selected_job = st.selectbox("Select Job ID", job_ids)
        new_status = st.selectbox("Select New Status", STATUSES)
        if st.button("Update Status"):
            for job in st.session_state.jobs:
                if job["Job ID"] == selected_job:
                    job["Status"] = new_status

                    # If marking COMPLETED, generate result if not already present
                    if new_status == "COMPLETED":
                        if not job["Result"]:
                            try:
                                simulator = simulators.get(job["Device"], AerSimulator())
                                result = simulator.run(job["Circuit"], shots=1024).result()
                                job["Result"] = result.get_counts(job["Circuit"])
                            except Exception as e:
                                job["Result"] = {}
                                st.warning(f"Failed to generate result for Job {selected_job}: {e}")

                    # If marking FAILED, clear the result
                    elif new_status == "FAILED":
                        job["Result"] = None

                    st.success(f"Job {selected_job} updated to {new_status}")

        st.subheader("Job Details & Circuit Visualization")
        for idx, job_info in enumerate(st.session_state.jobs):
            job_id = job_info["Job ID"]
            with st.expander(f"Job ID: {job_id} | User: {job_info['User']} | Device: {job_info['Device']} | Status: {job_info['Status']}"):
                st.write(f"Submitted At: {job_info['Submitted At']}")
                st.write(f"Status: {job_info['Status']}")
                vis_mode = st.radio("Circuit View Mode", ["mpl","text"], key=f"admin_{job_id}_{idx}_{time.time()}")
                if vis_mode=="mpl":
                    st.pyplot(job_info["Circuit"].draw('mpl'))
                else:
                    st.text(job_info["Circuit"].draw('text'))

                # Display graphs if result exists
                if job_info["Result"]:
                    st.write("Measurement Histogram:")
                    st.pyplot(plot_histogram(job_info["Result"]))
                    st.write("Measurement Counts Bar Graph:")
                    plot_counts_graph(job_info["Result"])


# ------------------------------
# PAGE: Search Jobs
# ------------------------------
elif st.session_state.page == "search_jobs":
    st.header("Search Jobs")
    with st.form("search_form"):
        search_job_id = st.text_input("Job ID")
        search_user = st.text_input("User")
        search_device = st.selectbox("Device", ["All"] + DEVICES)
        search_status = st.selectbox("Status", ["All"] + STATUSES)
        submitted = st.form_submit_button("Search")

    if submitted:
        filtered_jobs = [
            job for job in st.session_state.jobs
            if (search_job_id in job["Job ID"] or search_job_id == "")
            and (search_user in job["User"] or search_user == "")
            and (job["Device"] == search_device or search_device == "All")
            and (job["Status"] == search_status or search_status == "All")
        ]

        if filtered_jobs:
            df = pd.DataFrame(filtered_jobs)
            st.table(df.drop(columns=["Circuit", "Result"]))

            st.subheader("Job Details & Circuit Visualization")
            for idx, job_info in enumerate(filtered_jobs):
                job_id = job_info["Job ID"]
                with st.expander(f"Job ID: {job_id} | User: {job_info['User']} | Device: {job_info['Device']} | Status: {job_info['Status']}"):
                    st.write(f"Submitted At: {job_info['Submitted At']}")
                    st.write(f"Status: {job_info['Status']}")
                    vis_mode = st.radio("Circuit View Mode", ["mpl","text"], key=f"search_{job_id}_{idx}_{time.time()}")
                    if vis_mode=="mpl":
                        st.pyplot(job_info["Circuit"].draw('mpl'))
                    else:
                        st.text(job_info["Circuit"].draw('text'))
                    if job_info["Result"]:
                        st.write("Measurement Histogram:")
                        st.pyplot(plot_histogram(job_info["Result"]))
                        st.write("Measurement Counts Bar Graph:")
                        plot_counts_graph(job_info["Result"])
        else:
            st.info("No jobs found matching the search criteria.")

# ------------------------------
elif st.session_state.page == "bloch_spheres":
    st.header("🧠 Bloch Sphere Visualization for All Users' Jobs")

    # Include all users' completed statevector jobs
    matching_jobs = [
        job for job in st.session_state.jobs
        if job.get("Device") == "statevector_simulator" and job.get("Status") == "COMPLETED"
    ]

    if not matching_jobs:
        st.info("No completed jobs available for 'statevector_simulator'.")
    else:
        for idx, job in enumerate(matching_jobs):
            job_id = job.get("Job ID", "Unknown ID")
            user = job.get("User", "Unknown User")
            circuit = job.get("Circuit")

            if not circuit:
                st.warning(f"⚠️ Circuit data missing for Job ID: {job_id}")
                continue

            with st.expander(f"Job ID: {job_id} | User: {user}"):
                st.write(f"Submitted At: {job.get('Submitted At', 'N/A')}")

                # Render circuit
                try:
                    st.pyplot(circuit.draw("mpl"))
                except Exception as draw_err:
                    st.warning(f"⚠️ Error rendering circuit for Job {job_id}: {draw_err}")
                    continue

                st.subheader("🧠 Bloch Sphere Visualization for Each Qubit")
                try:
                    circ_copy = circuit.copy()
                    circ_copy.remove_final_measurements()
                    full_dm = DensityMatrix(circ_copy)

                    for i in range(circuit.num_qubits):
                        reduced_dm = partial_trace(full_dm, [q for q in range(circuit.num_qubits) if q != i])
                        bloch_vector = bloch_vector_from_dm(reduced_dm)
                        fig = plot_bloch_vector(bloch_vector, title=f"Qubit {i}")
                        st.pyplot(fig)

                except Exception as e:
                    st.warning(f"⚠️ Bloch sphere rendering failed for Job {job_id}: {e}")


# ------------------------------
# PAGE: Analytics & Comparison
# ------------------------------
elif st.session_state.page == "analytics_comparison":
    st.header("📊 Job Analytics & Comparison Dashboard")

    if not st.session_state.jobs:
        st.info("No jobs available for analytics.")
    else:
        df_jobs = pd.DataFrame(st.session_state.jobs)

        # ------------------------------
        # Job Status Pie Chart
        # ------------------------------
        st.subheader("Job Status Distribution")
        status_counts = df_jobs["Status"].value_counts()
        fig1, ax1 = plt.subplots()
        ax1.pie(status_counts, labels=status_counts.index, autopct='%1.1f%%', startangle=90,
                colors=['skyblue', 'lightgreen', 'salmon', 'gold'])
        ax1.axis('equal')
        st.pyplot(fig1)

        # ------------------------------
        # Jobs per Device
        # ------------------------------
        st.subheader("Jobs per Device")
        device_counts = df_jobs["Device"].value_counts()
        fig2, ax2 = plt.subplots()
        ax2.bar(device_counts.index, device_counts.values, color='orchid')
        ax2.set_xlabel("Device")
        ax2.set_ylabel("Number of Jobs")
        st.pyplot(fig2)

        # ------------------------------
        # Jobs per User
        # ------------------------------
        st.subheader("Jobs per User")
        user_counts = df_jobs["User"].value_counts()
        fig3, ax3 = plt.subplots()
        ax3.bar(user_counts.index, user_counts.values, color='lightcoral')
        ax3.set_xlabel("User")
        ax3.set_ylabel("Number of Jobs")
        st.pyplot(fig3)

        # ------------------------------
        # Filter Jobs for Comparison
        # ------------------------------
        st.subheader("Compare Jobs")
        users_list = ["All"] + sorted(df_jobs["User"].unique().tolist())
        devices_list = ["All"] + sorted(df_jobs["Device"].unique().tolist())
        statuses_list = ["All"] + STATUSES

        col1, col2, col3 = st.columns(3)
        with col1:
            filter_user = st.selectbox("Filter by User", users_list)
        with col2:
            filter_device = st.selectbox("Filter by Device", devices_list)
        with col3:
            filter_status = st.selectbox("Filter by Status", statuses_list)

        # Apply filters
        filtered_df = df_jobs.copy()
        if filter_user != "All":
            filtered_df = filtered_df[filtered_df["User"] == filter_user]
        if filter_device != "All":
            filtered_df = filtered_df[filtered_df["Device"] == filter_device]
        if filter_status != "All":
            filtered_df = filtered_df[filtered_df["Status"] == filter_status]

        if filtered_df.empty:
            st.info("No jobs match the selected filters for comparison.")
        else:
            st.subheader("Job Measurement Comparison")
            selected_jobs = st.multiselect("Select Jobs to Compare", filtered_df["Job ID"].tolist())

            if selected_jobs:
                comparison_df = filtered_df[filtered_df["Job ID"].isin(selected_jobs)]
                for idx, job_info in comparison_df.iterrows():
                    st.write(f"Job ID: {job_info['Job ID']} | User: {job_info['User']} | "
                             f"Device: {job_info['Device']} | Status: {job_info['Status']}")
                    st.write("Circuit:")
                    st.pyplot(job_info["Circuit"].draw('mpl'))

                    if job_info["Result"]:
                        st.write("Measurement Histogram:")
                        st.pyplot(plot_histogram(job_info["Result"]))
                        st.write("Measurement Counts Bar Graph:")
                        plot_counts_graph(job_info["Result"])

                # ------------------------------
                # Side-by-side Bloch Spheres (only for COMPLETED statevector jobs)
                # ------------------------------
                st.subheader("Bloch Sphere Visualization")
                for idx, job_info in comparison_df.iterrows():
                    if job_info["Status"] != "COMPLETED":
                        st.info(f"Bloch sphere not available for {job_info['Status']} job: {job_info['Job ID']}")
                        continue

                    if job_info["Device"] != "statevector_simulator":
                        st.info(f"Bloch sphere only available for 'statevector_simulator': {job_info['Job ID']}")
                        continue

                    circ_copy = job_info["Circuit"].copy()
                    circ_copy.remove_final_measurements()
                    try:
                        full_dm = DensityMatrix(circ_copy)
                        cols = st.columns(circ_copy.num_qubits)
                        for i in range(circ_copy.num_qubits):
                            reduced_dm = partial_trace(full_dm, [q for q in range(circ_copy.num_qubits) if q != i])
                            bloch_vector = bloch_vector_from_dm(reduced_dm)
                            fig_bloch = plot_bloch_vector(bloch_vector, title=f"Job {job_info['Job ID']} | Qubit {i}")
                            cols[i].pyplot(fig_bloch)
                    except Exception as e:
                        st.warning(f"⚠️ Bloch sphere rendering failed for Job {job_info['Job ID']}: {e}")



# ------------------------------
# PAGE: Entanglement Metrics
# ------------------------------
elif st.session_state.page == "entanglement_metrics":
    st.header("🧵 Qubit Entanglement Dashboard")

    completed_jobs = [job for job in st.session_state.jobs if job.get("Status") == "COMPLETED"]

    if not completed_jobs:
        st.info("No completed jobs available to analyze entanglement.")
    else:
        for job in completed_jobs:
            job_id = job.get("Job ID")
            circuit = job.get("Circuit")

            if not circuit:
                st.warning(f"⚠️ Circuit missing for Job {job_id}")
                continue

            st.subheader(f"Job ID: {job_id} | User: {job.get('User')}")

            try:
                # Make a copy and remove measurements
                circ_no_measure = circuit.remove_final_measurements(inplace=False)

                statevector = Statevector.from_instruction(circ_no_measure)
                density_matrix = DensityMatrix(statevector)

                purities = []
                for i in range(circuit.num_qubits):
                    reduced_dm = partial_trace(density_matrix, [q for q in range(circuit.num_qubits) if q != i])
                    purity = np.real(np.trace(reduced_dm.data @ reduced_dm.data))
                    purities.append(purity)

                st.write("Purity of each qubit:", purities)

                if circuit.num_qubits == 2:
                    # Concurrence between two qubits
                    concurrence_val = concurrence(density_matrix)
                    st.write(f"Concurrence (Qubit 0 & 1): {concurrence_val:.3f}")

                # Highlight highly entangled qubits
                entangled = ["Yes" if p < 0.95 else "No" for p in purities]
                df_ent = pd.DataFrame({"Qubit": list(range(circuit.num_qubits)), "Highly Entangled?": entangled, "Purity": purities})
                st.table(df_ent)

            except Exception as e:
                st.warning(f"⚠️ Entanglement calculation failed for Job {job_id}: {e}")

# ------------------------------
# PAGE: Job Replay
# ------------------------------
elif st.session_state.page == "job_replay":
    st.header("⏮️ Quantum Job History & Replay")

    if not st.session_state.jobs:
        st.info("No jobs available to replay.")
    else:
        job_options = [f"{job['Job ID']} | {job['User']}" for job in st.session_state.jobs]
        selected_job_str = st.selectbox("Select Job to Replay", job_options)
        selected_job = st.session_state.jobs[job_options.index(selected_job_str)]

        circuit = selected_job.get("Circuit")
        if not circuit:
            st.warning("⚠️ Selected job has no circuit data.")
        else:
            st.subheader(f"Job ID: {selected_job['Job ID']} | User: {selected_job['User']}")

            step = st.slider("Step through gates", 0, len(circuit.data), 0)
            qc_partial = QuantumCircuit(circuit.num_qubits, circuit.num_clbits)

            # Add gates up to the selected step
            for instr, qargs, cargs in circuit.data[:step]:
                qc_partial.append(instr, qargs, cargs)

            # Draw current state
            st.pyplot(qc_partial.draw("mpl"))

            # Show probabilities if simulation possible
            try:
                backend = AerSimulator()
                result = backend.run(qc_partial, shots=1024).result()
                counts = result.get_counts()
                st.write("Measurement Probabilities:")
                st.bar_chart(pd.DataFrame(counts.values(), index=counts.keys()))
            except Exception as e:
                st.warning(f"⚠️ Could not simulate circuit at step {step}: {e}")






Overwriting app.py


In [ ]:
import os
import time
import re

# Kill old processes
os.system('pkill -f streamlit || echo "No old streamlit"')
os.system('pkill -f cloudflared || echo "No old tunnel"')

# Run Streamlit in background (headless)
os.system('streamlit run app.py --server.port 8501 --server.headless true > streamlit.log 2>&1 &')

# Download Cloudflared and make executable
if os.system('wget -q -O cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64') != 0:
    print("⚠️ Failed to download Cloudflared")
os.system('chmod +x cloudflared')

# Start Cloudflare Tunnel
os.system('nohup ./cloudflared tunnel --url http://localhost:8501 --no-autoupdate > cloudflared.log 2>&1 &')

# Wait for URL in log
url = None
for i in range(60):  # wait up to 2 minutes
    time.sleep(10)
    try:
        log = open('cloudflared.log').read()
        urls = re.findall(r"https://[-0-9a-z]+\.trycloudflare\.com", log)
        if urls:
            url = urls[0]
            break
    except:
        continue

if url:
    print("🌍 Public Streamlit URL:", url)
else:
    print("⚠️ No URL found. Wait a few more seconds or re-run the cell.")


⚠️ Failed to download Cloudflared
🌍 Public Streamlit URL: https://himself-fate-dolls-gothic.trycloudflare.com
